<a href="https://colab.research.google.com/github/hBharish/Subtheme-Sentiment-Analysis/blob/main/Subtheme_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import pandas as pd
import numpy as np
from transformers import BertTokenizer
from sklearn.preprocessing import LabelEncoder
from re import escape
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification


In [ ]:
'''
# Load dataset
df = pd.read_csv('Evaluation-dataset.csv')
df = df.dropna(subset=[df.columns[1]])

# Initialize lists to store reviews and labels
reviews = []
unique_labels = set()
llb = []

# Iterate over each row in the dataset
for row in df.itertuples(index=False):
    temp_labels = []
    for item in row[1:]:
        if pd.notnull(item):
            if "positive" in item.lower() or "negative" in item.lower():
                unique_labels.add(item.lower())
                temp_labels.append(item.lower())
    if temp_labels:
        reviews.append(row[0])
        llb.append(temp_labels)

# Convert labels set to list
unique_labels = list(unique_labels)
print(len(reviews))

# Create DataFrame with reviews and unique labels
df_new = pd.DataFrame(columns=['reviews'] + unique_labels)

# Set values in DataFrame based on labels presence
for review, labels in zip(reviews, llb):
    row_values = [1 if label in labels else 0 for label in unique_labels]
    df_new.loc[len(df_new)] = [review] + row_values

# Save DataFrame to CSV
df_new.to_csv('reviews_with_labels.csv', index=False)

print("CSV file saved successfully.")'''

In [3]:
data = pd.read_csv("reviews_with_labels.csv")


In [ ]:

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=62,problem_type="multi_label_classification")


In [5]:
model = model.to('cuda')


In [6]:
reviews = list(data["reviews"])
labels = data.iloc[:, 1:].values
X_train, X_val, y_train, y_val = train_test_split(reviews, labels, test_size=0.2)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512,return_tensors='pt')
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512,return_tensors='pt')


In [7]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels is not None:
            item["labels"] = torch.tensor(self.labels[idx]).float()
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)


In [8]:
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    labels = np.argmax(labels, axis=1)
    accuracy = accuracy_score(labels, pred)
    recall = recall_score(labels, pred, average='weighted')
    precision = precision_score(labels, pred, average='weighted')
    f1 = f1_score(labels, pred, average='weighted')
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [13]:

args = TrainingArguments(
    output_dir="output",
    num_train_epochs=10,
    per_device_train_batch_size=20,

)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,

)


In [ ]:
trainer.train()



In [ ]:
trainer.evaluate()

In [26]:

text = ("Great service and price from [REDACTED] let down by fitters. [REDACTED] price was by far the best I could find. "
        "While they delivered my tyres on time, their fitting garage had suffered an equipment failure and had to delay "
        "the fitting by three days. This is the only reason for me dropping my rating to four stars.")
inputs = tokenizer(text, padding=True, truncation=True, return_tensors='pt').to('cuda')
outputs = model(**inputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
predictions = predictions.cpu().detach().numpy()
high_prob_indices = [i for i, prob in enumerate(predictions[0]) if prob > 0.020]
predicted_labels = [lbl[i] for i in high_prob_indices]
output_labels = ', '.join(predicted_labels)
print(output_labels)


change of date negative, value for money positive, delivery punctuality positive, location positive, length of fitting positive, mobile fitter positive, ease of booking positive, garage service positive
